# Estimating Work Tour Scheduling

This notebook illustrates how to re-estimate the mandatory tour scheduling component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd

We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [2]:
os.chdir('test')

# Load data and prep model for estimation

In [3]:
modelname = "mandatory_tour_scheduling_work"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

# Review data loaded from the EDB

The next (optional) step is to review the EDB, including the coefficients, utilities specification, and chooser and alternative data.

## Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_dummy,1.000000,T
coef_free_flow_round_trip_auto_time_shift_effects_departure,-0.001140,F
coef_free_flow_round_trip_auto_time_shift_effects_duration,0.002210,F
coef_part_time_worker_departure_shift_effects,0.067360,F
coef_non_working_adult_duration_shift_effects,-0.120700,F
...,...,...
coef_duration_constants_9_hours,0.055706,F
coef_duration_constants_10_hours,0.000000,T
coef_duration_constants_11_hours,-0.347795,F


## Utility specification

In [5]:
data.spec

,Label,Description,Expression,Coefficient
0,util_free_flow_round_trip_auto_time_shift_effe...,Free-flow round trip auto time shift effects -...,roundtrip_auto_time_to_work * start,coef_free_flow_round_trip_auto_time_shift_effe...
1,util_free_flow_round_trip_auto_time_shift_effe...,Free-flow round trip auto time shift effects -...,roundtrip_auto_time_to_work * duration,coef_free_flow_round_trip_auto_time_shift_effe...
2,util_part_time_worker_departure_shift_effects,Part-time worker departure shift effects,(ptype == 2) * start,coef_part_time_worker_departure_shift_effects
3,util_non_working_adult_duration_shift_effects,Non-working adult duration shift effects,(ptype == 4) * duration,coef_non_working_adult_duration_shift_effects
4,util_university_student_departure_shift_effects,University student departure shift effects,(ptype == 3) * start,coef_university_student_departure_shift_effects
...,...,...,...,...
60,util_duration_constants_9_hours,Duration Constants -- 9 hours,duration == 9,coef_duration_constants_9_hours
61,util_duration_constants_10_hours,Duration Constants -- 10 hours,duration == 10,coef_duration_constants_10_hours
62,util_duration_constants_11_hours,Duration Constants -- 11 hours,duration == 11,coef_duration_constants_11_hours
63,util_duration_constants_12_to_13_hours,Duration Constants -- 12 to 13 hours,(duration > 11) & (duration < 14),coef_duration_constants_12_to_13_hours


## Chooser data

In [6]:
data.chooser_data

,tour_id,model_choice,override_choice,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,...,home_is_rural,mandatory_tour_frequency,is_worker,is_student,is_university,workplace_zone_id,school_zone_id,home_zone_id,start_previous,end_previous
0,2961920,32,35,72241,work,1,1,1,1,mandatory,...,False,work1,True,False,False,9,-1,17,5,5
1,2970120,51,51,72441,work,1,1,1,1,mandatory,...,False,work1,True,False,False,100,-1,60,5,5
2,2998943,29,28,73144,work,1,1,1,1,mandatory,...,False,work1,True,False,False,77,-1,125,5,5
3,3013252,47,47,73493,work,1,1,1,1,mandatory,...,False,work1,True,False,False,117,-1,133,5,5
4,3015794,59,59,73555,work,1,1,1,1,mandatory,...,False,work1,True,False,False,167,-1,136,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,308070309,184,184,7513909,work,2,2,2,2,mandatory,...,False,work2,True,False,False,11,-1,21,17,21
2120,308073875,41,38,7513996,work,1,1,1,1,mandatory,...,False,work1,True,False,False,67,-1,21,5,5
2121,308090603,27,27,7514404,work,1,1,1,1,mandatory,...,False,work1,True,False,False,106,-1,81,5,5
2122,308105896,6,6,7514777,work,1,1,1,1,mandatory,...,False,work1,True,True,True,87,185,106,5,5


## Alternatives data

In [7]:
data.alt_values

,tour_id,variable,0,1,2,3,4,5,6,7,...,180,181,182,183,184,185,186,187,188,189
0,2961920,duration,0,1,2,3,4,5,6,7,...,0,1,2,3,0,1,2,0,1,0
1,2961920,end,5,6,7,8,9,10,11,12,...,20,21,22,23,21,22,23,22,23,23
2,2961920,in_period,EA,AM,AM,AM,AM,MD,MD,MD,...,EV,EV,EV,EV,EV,EV,EV,EV,EV,EV
3,2961920,mode_choice_logsum,-0.6808670711507444,-0.9609052826735235,-0.9609052826735235,-0.9609052826735235,-0.9609052826735235,-1.1106688167107288,-1.1106688167107288,-1.1106688167107288,...,-0.8117834942924501,-0.8117834942924501,-0.8117834942924501,-0.8117834942924501,-0.8117834942924501,-0.8117834942924501,-0.8117834942924501,-0.8117834942924501,-0.8117834942924501,-0.8117834942924501
4,2961920,out_period,EA,EA,EA,EA,EA,EA,EA,EA,...,EV,EV,EV,EV,EV,EV,EV,EV,EV,EV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146551,308122624,util_subsequent_tour_must_start_after_previous...,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
146552,308122624,util_tours_by_student_duration_lt_8_hrs,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
146553,308122624,util_tours_by_worker_duration_lt_8_hrs,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
146554,308122624,util_university_student_departure_shift_effects,5,5,5,5,5,5,5,5,...,20,20,20,20,21,21,21,22,22,23


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [8]:
model.estimate()

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction,0.326676,0.362700,0.0,-25.0,25.0,0,,0.326676
coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction,0.298901,-0.101200,0.0,-25.0,25.0,0,,0.298901
coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction,-0.038388,0.177100,0.0,-25.0,25.0,0,,-0.038388
coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction,-0.012605,-0.212300,0.0,-25.0,25.0,0,,-0.012605
coef_arrival_constants_am_peak,-2.527138,-1.854521,0.0,-25.0,25.0,0,,-2.527138
...,...,...,...,...,...,...,...,...
coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs,5.443390,2.582000,0.0,-25.0,25.0,0,,5.443390
coef_subsequent_tour_must_start_after_previous_tour_ends,-100.000000,-100.000000,0.0,-25.0,25.0,1,,-100.000000
coef_tours_by_student_duration_lt_8_hrs,20.927636,2.582000,0.0,-25.0,25.0,0,,20.927636
coef_tours_by_worker_duration_lt_8_hrs,19.258236,0.912600,0.0,-25.0,25.0,0,,19.258236


<ipython-input-8-5091d419e17b>:1: PossibleOverspecification: WARNING: Model is possibly over-specified (hessian is nearly singular).
  model.estimate()
/Users/jeffnewman/OneDrive - Cambridge Systematics/Git/larch/larch/linalg/__init__.py:18: UserWarning: minimum eig 2.580800976238575e-22 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣                x: coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction              0.326676
┃                   coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction        0.298901
┃                   coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction          -0.038388
┃                   coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction    -0.012605
┃                   coef_arrival_constants_am_peak                                                         -2.527138
┃                                                                                                            ...    
┃                   coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs                                   5.443390
┃                   coef_subsequent_tour_must_start_after_previous_tour_ends                              -25.000000
┃                   coef_tours_by_student_duration_lt_8_hrs                                                20.927636
┃                   coef_tours_by_worker_duration_lt_8_hrs                                                 19.258236
┃                   coef_university_student_departure_shift_effects                                         0.076480
┃                   Length: 63, dtype: float64
┣          loglike: -8399.82196783851
┣        d_loglike: coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction             6.227709e-05
┃                   coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction      -3.094051e-04
┃                   coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction          2.047044e-04
┃                   coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction    1.044967e-04
┃                   coef_arrival_constants_am_peak                                                        -2.094941e-05
┃                                                                                                              ...     
┃                   coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs                                 -2.124468e-04
┃                   coef_subsequent_tour_must_start_after_previous_tour_ends                               0.000000e+00
┃                   coef_tours_by_student_duration_lt_8_hrs                                               -7.862668e-16
┃                   coef_tours_by_worker_duration_lt_8_hrs                                                -7.862668e-16
┃                   coef_university_student_departure_shift_effects                                       -1.306805e-02
┃                   Length: 63, dtype: float64
┣              nit: 64
┣             nfev: 177
┣             njev: 64
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=8, microseconds=813504)
┣           method: 'slsqp'
┣          n_cases: 2124
┣ iteration_number: 64
┣          logloss: 3.954718440601935

### Estimated coefficients

In [9]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction,0.327,0.208,1.57,,0.00,
coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction,0.299,0.244,1.23,,0.00,
coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction,-0.0384,0.207,-0.19,,0.00,
coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction,-0.0126,0.274,-0.05,,0.00,
coef_arrival_constants_am_peak,-2.53,0.539,-4.68,***,0.00,
coef_arrival_constants_early,0.00,NA,NA,,0.00,fixed value
coef_arrival_constants_evening,-0.0858,0.208,-0.41,,0.00,
coef_arrival_constants_late,-1.22,0.282,-4.33,***,0.00,
coef_arrival_constants_midday_1,-0.490,0.186,-2.64,**,0.00,
coef_arrival_constants_midday_2,-0.383,0.132,-2.90,**,0.00,


# Output Estimation Results

In [10]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [11]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [12]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_dummy,1.000000,T
1,coef_free_flow_round_trip_auto_time_shift_effe...,0.000599,F
2,coef_free_flow_round_trip_auto_time_shift_effe...,0.004216,F
3,coef_part_time_worker_departure_shift_effects,0.095094,F
4,coef_non_working_adult_duration_shift_effects,-0.120700,F
...,...,...,...
59,coef_duration_constants_9_hours,0.137251,F
60,coef_duration_constants_10_hours,0.000000,T
61,coef_duration_constants_11_hours,-0.358479,F
62,coef_duration_constants_12_to_13_hours,-1.132000,F
